In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import random


In [4]:
# Función para guardar los datos en un archivo JSON
def guardar_en_json(datos, nombre_archivo):
    try:
        with open(nombre_archivo, mode='w', encoding='utf-8') as archivo:
            # Escribir los datos en formato JSON, con indentación para mayor legibilidad
            json.dump(datos, archivo, ensure_ascii=False, indent=4)
    except Exception as e:
        print(f"Error al guardar el archivo JSON: {e}")



# Convertir los datos a un DataFrame de pandas
def convertir_a_dataframe(datos):
    df = pd.DataFrame([datos])
    return df


# Crear un DataFrame vacío
df_relojes = pd.DataFrame()

def parsear_relojes(links):
    global df_relojes  # Acceder al DataFrame global
    for link in links:
        url = f"https://www.chrono24.es{link}"
        browser = webdriver.Edge()

        try:
            # Cargar la página del reloj
            browser.get(url)
            time.sleep(random.randint(5, 10))  # Pausa para evitar bloqueos
            wait = WebDriverWait(browser, 10)

            # Si aparece un popup, cerrarlo
            try:
                wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/dialog/div/div[2]/button"))).click()
            except Exception:
                pass  # Si no hay popup, continuar

            # Esperar a que la sección de detalles esté presente
            wait.until(EC.presence_of_element_located((By.XPATH, "//div[@role='tabpanel']")))

            # Obtener el HTML de la página
            html = browser.page_source
            soup = BeautifulSoup(html, "html.parser")

            # Buscar la tabla de características
            tabla = soup.find("div", {"role": "tabpanel"})
            if tabla:
                filas = tabla.find_all("tr")
                detalles = {}

                for fila in filas:
                    celdas = fila.find_all("td")
                    if len(celdas) >= 2:  # Procesar solo filas con pares clave-valor
                        clave = celdas[0].text.strip()
                        valor = celdas[1].text.strip()
                        # Limpiar espacios no rompibles
                        valor = valor.replace("\xa0", " ")
                        detalles[clave] = valor

                # Agregar los datos extraídos al DataFrame
                df_relojes = pd.concat([df_relojes, pd.DataFrame([detalles])], ignore_index=True)

                # Guardar los datos en un archivo JSON
                with open("datos_relojes.json", "w", encoding="utf-8") as archivo_json:
                    df_relojes.to_json(archivo_json, orient="records", force_ascii=False, indent=4)

                # Mostrar los datos extraídos
                print("Datos extraídos del reloj:")
                for clave, valor in detalles.items():
                    print(f"{clave}: {valor}")

            else:
                print("No se encontró el elemento 'tabpanel'.")

        except Exception as e:
            print(f"Error general al procesar el reloj {link}: {e}")

        finally:
            # Cerrar el navegador
            browser.quit()




# DATA DE UN RELOJ

In [64]:
# URL del reloj en Chrono24
url = "https://www.chrono24.es/rolex/rolex-datejust-41--id37077875.htm?searchHash=e7a6168c_looecP&pos=1"
browser = webdriver.Edge()

try:
    # Cargar la página
    browser.get(url)
    wait = WebDriverWait(browser, 10)

    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/dialog/div/div[2]/button"))).click()
    except Exception:
        pass  

    wait.until(EC.presence_of_element_located((By.XPATH, "//div[@role='tabpanel']")))

    # Obtener el HTML de la página
    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")

    tabla = soup.find("div", {"role": "tabpanel"})
    if tabla:
        filas = tabla.find_all("tr")
        resultados = {}

        for fila in filas:
            celdas = fila.find_all("td")
            if len(celdas) >= 2:  # Solo procesar filas con pares clave-valor
                clave = celdas[0].text.strip()
                valor = celdas[1].text.strip()
                # Eliminar los espacios no rompibles (representados por '\xa0')
                valor = valor.replace("\xa0", " ")
                resultados[clave] = valor

        # Guardar los resultados en un archivo JSON
        guardar_en_json(resultados, 'datos_reloj.json')
        # Convertir los resultados en un DataFrame de pandas
        df = convertir_a_dataframe(resultados)

    else:
        print("No se encontró el elemento 'tabpanel'.")

except Exception as e:
    print("Error general:", e)

finally:
    browser.quit()

df


Datos guardados correctamente en 'datos_reloj.json'.


,Código del anuncio,Marca,(Reloj) Modelo,Número de referencia,Calibre,Material de la caja,Material de la pulsera,Año de fabricación,Estado,Contenido de la entrega,...,Número de joyas,Diámetro,Resistente al agua,Material del bisel,Cristal,Esfera,Esfera con números,Color de la pulsera,Cierre,Material del cierre
0,M2PGZ3,Rolex,Datejust 41,126334,3235,Oro blanco,Acero,2022,Usado (Muy bueno)\n\n E...,"Con estuche original, con documentos originales",...,31,41 mm\n\nProbar,10 ATM,Oro blanco,Cristal de zafiro,Verde,Sin cifras,Acero,Cierre desplegable,Acero


# RECORRER LA TABLA 

In [5]:
links = []

# URL de toda la pagina
url = "https://www.chrono24.es/rolex/index.htm"
browser = webdriver.Edge()

try:
    # Cargar la página
    browser.get(url)
    time.sleep(random.randint(10,15))
    wait = WebDriverWait(browser, 10)

    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/dialog/div/div[2]/button"))).click()
    except Exception:
        pass  

    wait.until(EC.presence_of_element_located((By.ID, "wt-watches")))

    # Obtener el html de la pagina
    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")

    tabla = soup.find("div", {"id" : "wt-watches"})
    if tabla:
        anuncio = tabla.find_all("a")
        for reloj in anuncio:
            link = reloj.get("href")
            if link:
                links.append(link)
    else:
        print("No se encontro el elemento 'wt-watches'")

except Exception as e:
    print(f"Error general: {e}")

finally:
    browser.quit()

links

['/rolex/rolex-yacht-master--id35039453.htm',
 '/rolex/rolex-oyster-perpetual-date--id35646398.htm',
 '/rolex/rolex-submariner--id35973403.htm',
 '/rolex/rolex-gmt-master-ii-16710-pepsi-no-holes--id34829777.htm',
 '/rolex/rolex-daytona-cosmograph-steel--gold-y-serial-2002--id37078947.htm',
 '/rolex/submariner-date--id33784145.htm',
 '/rolex/daytona-full-set-rrr--id37822582.htm',
 '/rolex/rolex-like-new-condition-2022-gmt-master-ii--id37703813.htm',
 '/rolex/datejust-36-steel--gold-black-diamond-dial--id37754998.htm',
 '/rolex/rolex-oyster-perpetual-date-automatic-steel--18k-gold-ladies-ref6917--id22329306.htm',
 '/rolex/rolex-submariner-date-patina-dial-ref-16610-from-1995--id36967126.htm',
 '/rolex/daytona--id34768417.htm',
 '/rolex/rolex-lady-datejust--id35978010.htm',
 '/rolex/rolex-oyster-perpetual-41-like-new-99-2022-124300--id36288055.htm',
 '/rolex/rolex-datejust-36--id37372974.htm',
 '/rolex/rolex-datejust-36-linen-dial--id35972519.htm',
 '/rolex/day-date-president--id37565200.

In [6]:
df_relojes = pd.DataFrame()
parsear_relojes(links)

Datos extraídos del reloj:
Código del anuncio: KV0M57
Marca: Rolex
(Reloj) Modelo: Yacht-Master
Número de referencia: 169622
(Yacht-Master 29mm)
Calibre: 2235
Material de la caja: Acero
Material de la pulsera: Acero
Año de fabricación: 2005 (Año aproximado)
Estado: Usado (Bueno)

                        El artículo presenta señales visibles y perceptibles de uso, como arañazos, rozaduras o pequeñas abolladuras.
Contenido de la entrega: Sin estuche original, sin documentos originales
Género: Reloj de dama
Ubicación: España, Valencia
Precio: 4450 € [Sujeto a negociación]
Disponibilidad: Artículo en stock
Reserva de marcha: 48 h
Número de joyas: 31
Diámetro: 29 mm

Probar
Resistente al agua: 10 ATM
Material del bisel: Platino
Cristal: Cristal de zafiro
Esfera: Plata
Esfera con números: Sin cifras
Color de la pulsera: Acero
Ancho de la pulsera: 12 mm

Asistente para determinar tamaño
Cierre: Cierre desplegable
Material del cierre: Acero
Ancho de cierre: 13 mm
Datos extraídos del reloj:
Cód

: 

In [3]:
df_relojes = pd.DataFrame()
df_relojes

NameError: name 'pd' is not defined